# Transformers

&emsp; Transformers, a game-changing development in machine learning, emerged in 2017 with the paper **"Attention Is All You Need"** by Vaswani et al. These models have notably impacted the field, excelling in natural language processing **(NLP)** tasks like **translation**, **sentiment analysis**, and **text summarization**. Famous Transformers include **BERT**, **GPT-3**, and **RoBERTa**.

Here are some of the key features of Transformers that make them so powerful:

- Transformers effectively **capture long-range correlation** in the data, a challenge for RNNs.
- They employ **self-attention mechanisms** to process and contextualize information across the entire sequence.
- Unlike RNNs, Transformers are **parallelizable** and thus more efficient to train.

In [ ]:
# defaults imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from mnist import Train, Val

import numpy as np
from tqdm import tqdm, trange
from time import time, sleep
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, top_k_accuracy_score

In [ ]:
# Some constants
repeat = 1_000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x = torch.randn(256, 28, 28).to(DEVICE)
DEVICE

## Key Concepts and Components

&emsp; Transformers are composed of several key components, including **self-attention**, **multi-head attention**, **masked attention**, **positional encoding** etc. These components are stacked together to form the Transformer architecture.

<!-- ![Transformer Architecture](trying_to_make_transformer.svg) -->

### Vanilla Self-Attention Mechanism

Self-attention is the basic building block of transformer models. Let us first understand the concept of self-attention without any learnable weights.

&emsp; Let us take the example of some NLP task to understand the concept of self-attention. Let us have a sentences of length $n$. Also for each word in the sentence, we have a vector of length $k$. So this sentence can be represented as a matrix of size ($n \times k$). Let us call this matrix $X_{n \times k}$. <span style="font-size: 12px">(we are not considering the batch dimension for simplicity)</span>

#### Step 1:

&emsp; We multiply $X$ with its transpose to get a matrix of size ($n \times n$). Let us call this matrix $W_{n\times n}$. This operation can be thought of as a similarity measure between each word in the sentence with every other word in the sentence. The diagonal elements of $W_{n\times n}$ will be the similarity of each word with itself. The off-diagonal elements will be the similarity of each word with every other word in the sentence.

$$
W_{n\times n} := X_{n \times k} \times X_{k \times n}^T
$$

**How come the product of the two word embeddings gives the similarity between the two words?** (cosine similarity is equal to the dot product of two unit vectors)



#### Step 2:

&emsp; In this step, we scale the result by $\frac{1}{\sqrt{k}}$ ([why?](error_proof.ipynb)), and  apply a softmax function to each column of $W_{n\times n}$. This step can be thought of as a way to normalize the similarity scores. After completing this step, we get an updated $W_{n\times n}$ matrix.

$$
W_{n\times n} := \text{softmax}\left(\frac{W_{n\times n}}{\sqrt{k}}\right)
$$

#### Step 3:

&emsp; In this step, we multiply the updated $W_{n\times n}$ with $X_{n \times k}$ to get a matrix of size ($n \times k$). Let us call this matrix $Y_{n \times k}$. This step can be thought of as a weighted sum of the vectors in $X_{n \times k}$, where the weights are the values in $W_{n\times n}$. Thus as the new matrix $Y_{n \times k}$, we have a matrix which has the same shape as $X_{n \times k}$, but instead of telling about each word in the sentence, it tells about the importance of each word in the sentence in the context of every other word in the sentence.

$$
Y_{n \times k} := W_{n\times n} \times X_{n \times k}
$$

In [ ]:
class VanillaSelfAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        # size of x is expected to be [batch_size, seq_len=n, embedding_dim=k]
        w = x @ x.transpose(-1, -2)  # w.size() = [batch_size, n, n]
        w = w / torch.sqrt(torch.tensor(x.shape[-1]).float())  # w / sqrt(k)
        w = F.softmax(w, dim=-1)
        y = w @ x  # y.size() = [batch_size, n, k]
        return y

### Self-Attention with Weights

Now, as we know the concept of self Attention, we can extend it to self-attention with weights. In this case, we have three weight matrices $W^Q$, $W^K$, and $W^V$ all of shapes ($k \times d$). Here $d$ is the output dimension. The next calculation is similar to the previous one and is shown below.

$Q_{n \times d} := X_{n \times k} \times W_{k \times d}^Q$<br>
$K_{n \times d} := X_{n \times k} \times W_{k \times d}^K$<br>
$V_{n \times d} := X_{n \times k} \times W_{k \times d}^V$

$W_{n\times n}  := Q_{n\times d} \times K_{d \times n}^T$<br>
$W_{n\times n}  := \text{softmax}\left(\frac{W_{n\times n}}{\sqrt{d}}\right)$<br>
$Y_{n \times d} := W_{n\times n} \times V_{n \times d}$

Note that, in case of self-attention without weights, the shape of $Y$ remained same as $X$ both ($n \times k$). But in case of self-attention with weights, the shape of $X$ is ($n \times k$) and the shape of $Y$ is ($n \times d$). It is as if, the embedding dimension of each word in the sentence has been changed from $k$ to $d$.

In [ ]:
# Without using linear layers

class SelfAttention(nn.Module):
    def __init__(self, input_dim, output_dim=None):
        super().__init__()
        if output_dim is None: output_dim = input_dim
        Wq = torch.empty(input_dim, output_dim)
        Wk = torch.empty(input_dim, output_dim)
        Wv = torch.empty(input_dim, output_dim)

        torch.nn.init.xavier_uniform_(Wq)
        torch.nn.init.xavier_uniform_(Wk)
        torch.nn.init.xavier_uniform_(Wv)

        self.Wq = nn.Parameter(Wq)
        self.Wk = nn.Parameter(Wk)
        self.Wv = nn.Parameter(Wv)

    def forward(self, x):
        # sixe of x is expected to be [batch_size, seq_len=n, input_dim=k]
        Q = x @ self.Wq  # q.size() = [batch_size, n, output_dim=d]
        K = x @ self.Wk  # k.size() = [batch_size, n, output_dim=d]
        V = x @ self.Wv  # v.size() = [batch_size, n, output_dim=d]

        w = Q @ K.transpose(-1, -2)  # w.size() = [batch_size, n, n]
        w = w / torch.sqrt(torch.tensor(x.shape[-1]).float())
        w = F.softmax(w, dim=-1)
        y = w @ V
        return y

### Multi-Head Attention

In multi-head attention, we have multiple self-attention layers in parallel. Each of these layers has its own set of weights. The output of each of these layers is concatenated and then passed through a linear layer. The output of this linear layer is the final output of the multi-head attention layer. 

Suppose there are $h$ heads in the multi-head attention layer, and we take d as the output dimension for the multi-head attention layer. Incase we use $d$ as the output dimension for each head, the output from each heads will be of shape ($n \times d$). The output of all these heads concatenated will be of shape ($n \times d*h$). But we wanted the output of the multi-head attention layer to be of shape ($n \times d$). So, instead of using $d$ as the output dimension for each head, we use $\frac{d}{h}$ as the output dimension for each head. Thus the output of each head will be of shape ($n \times \frac{d}{h}$). The output of all these heads concatenated will be of shape ($n \times d$). <span style="font-size: 12px">(assuming h divides d perfectly)</span>

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        assert output_dim % num_heads == 0, f"{output_dim = } must be divisible by {num_heads = }"
        self.head_dim = output_dim // num_heads

        # nothing fancy, just initializing the weights
        Wq = torch.empty(input_dim, output_dim)
        Wk = torch.empty(input_dim, output_dim)
        Wv = torch.empty(input_dim, output_dim)
        torch.nn.init.xavier_uniform_(Wq)
        torch.nn.init.xavier_uniform_(Wk)
        torch.nn.init.xavier_uniform_(Wv)
        self.Wq = nn.Parameter(Wq)
        self.Wk = nn.Parameter(Wk)
        self.Wv = nn.Parameter(Wv)

        self.fc_out = nn.Linear(output_dim, output_dim)

    def forward(self, x):
        # print(x.size())
        batch_size, seq_len, _ = x.size()

        Q = x @ self.Wq
        K = x @ self.Wk
        V = x @ self.Wv

        Q = Q.reshape(batch_size, seq_len, self.num_heads, self.head_dim)
        K = K.reshape(batch_size, seq_len, self.num_heads, self.head_dim)
        V = V.reshape(batch_size, seq_len, self.num_heads, self.head_dim)

        w = torch.einsum("nqhd,nkhd->nhqk", [Q, K])
        w = w / torch.sqrt(torch.tensor(self.head_dim).float())
        w = F.softmax(w, dim=-1)
        y = torch.einsum("nhql,nlhd->nqhd", [w, V])
        y = y.reshape(batch_size, seq_len, -1)  # [batch_size, seq_len, output_dim]

        out = self.fc_out(y)

        return out

### Masked Attention

As explained in step 2 of the vanilla self-attention mechanism, we scale the matrix $W_{n\times n}$ by $\frac{1}{\sqrt{k}}$ and then apply a softmax function to each column of $W_{n\times n}$. Here between scaling down the matrix and applying the softmax function, we do the masking.

**Motivation:** In the case of language translation, we do not want the model to look at the words that come after the word that we are trying to translate. (For example, if we are trying to predict the next part of the sentence "I am going to the", we do not want the model to look at the words "childrens park" as it comes after the word "the". So we mask the words that come after the word that we are trying to predict. Now, in the next go, we will show it the word "childrens", and would expect it to predict just the word "park". So we mask the words that come after the word "childrens". This is called masking.)

**How do we do masking?** We make the upper traiangular part of the matrix $W_{n\times n}$ equal to $-\infty$. This is done so that when we apply the softmax function, the values in the upper triangular part of the matrix become zero, and the the full probability mask is distributed only among the lower triangular part of the matrix.

In [ ]:
class MaskedMultiHeadAttention(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        assert output_dim % num_heads == 0, f"{output_dim = } must be divisible by {num_heads = }"
        self.head_dim = output_dim // num_heads

        # nothing fancy, just initializing the weights
        Wq = torch.empty(input_dim, output_dim)
        Wk = torch.empty(input_dim, output_dim)
        Wv = torch.empty(input_dim, output_dim)
        torch.nn.init.xavier_uniform_(Wq)
        torch.nn.init.xavier_uniform_(Wk)
        torch.nn.init.xavier_uniform_(Wv)
        self.Wq = nn.Parameter(Wq)
        self.Wk = nn.Parameter(Wk)
        self.Wv = nn.Parameter(Wv)

        self.fc_out = nn.Linear(output_dim, output_dim)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        Q = x @ self.Wq
        K = x @ self.Wk
        V = x @ self.Wv

        Q = Q.reshape(batch_size, seq_len, self.num_heads, self.head_dim)
        K = K.reshape(batch_size, seq_len, self.num_heads, self.head_dim)
        V = V.reshape(batch_size, seq_len, self.num_heads, self.head_dim)

        w = torch.einsum("nqhd,nkhd->nhqk", [Q, K])
        w = w / torch.sqrt(torch.tensor(self.head_dim).float())

        mask = torch.tril(torch.ones(seq_len, seq_len)) == 0
        mask = mask.to(self.Wq.device)
        w = w.masked_fill(mask, float("-inf"))

        w = F.softmax(w, dim=-1)
        y = torch.einsum("nhql,nlhd->nqhd", [w, V])
        y = y.reshape(batch_size, seq_len, -1)  # [batch_size, seq_len, output_dim]

        out = self.fc_out(y)

        return out

### Positional Encoding

Adhil has already explained this in detail, I am not going to repeat the whole here, but I will just give a brief overview of the concept.

- Positional embedding vector is a vector of same shape as the input embeding vector and it's values don't change with the input.
- It carries information about the position of the word in the sentence.
- We element-wise add the positional embedding vector to the input embedding vector to get the final embedding vector for each word in the sentence.
- Thus we get a new embedding vector which carries information about the word itself as well as its position in the sentence.

## Transformer Architecture

&emsp; We would build a small transformer model to understand the architecture of the transformer. The transformer model that we are going to build is shown below.

<!-- ![Transformer Block](transformer_block.png) -->
<img src="transformer_block.png" alt="transformer block" width="200"/>

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads=8, dropout=0.1, forward_expansion=1):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.dropout = nn.Dropout(dropout)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, int(forward_expansion * embed_size)),
            nn.ReLU(),
            nn.Linear(int(forward_expansion * embed_size), embed_size),
        )

        self.norm2 = nn.LayerNorm(embed_size)

    def forward(self, x):
        attention = self.attention(x)
        x = self.dropout(self.norm1(attention + x))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

class Model(nn.Module):
    def __init__(self, embed_size=28, num_classes=10, heads=4, forward_expansion=1, dropout=0.1):
        super(Model, self).__init__()
        self.embed_size = embed_size
        self.num_classes = num_classes

        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
        self.fc_out = nn.Linear(embed_size*embed_size, num_classes)

    def forward(self, x):
        out = self.transformer_block(x)
        out = out.flatten(start_dim=1)
        out = self.fc_out(out)
        return F.softmax(out, dim=-1)